# 07-2 심층 신경망
### 인공 신경망에 층을 여러 개 추가하여 심층 신경망을 만드는 방법
---
</br>

💡용어 정리💡  
❓ 인공신경망의 강점?? ➡️ 층을 추가 -> 입력 데이터에 대한 연속적인 학습  
❓ 렐루 함수 ➡️ 입력이 양수일 경우 입력 출력, 음수일 경우 0 출력  
 --> 심층 신경망 중 이미지 처리에 특히 적합함!  


### 1) 케라스 API를 이용하여 패션 MNIST 데이터셋 로딩

In [1]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


### 2) 데이터 변환 및 재정렬, 훈련 데이터/테스트 데이터로 분할
**[29]에 작성된 코드 사용!! Flatten 층 사용**

In [2]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 # 픽셀값 0~255 --> 0~1 변환
# Flattern 층 으로 대체 가능!! (밑에 코드로 설명)
train_scaled = train_scaled.reshape(-1, 28*28) # 28x28 크기의 2차원 배열 --> 784 크기의 1차원으로 변환

train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

**은닉층과 출력층 생성(1)**
- 첫 번째 층에는 반드시 input_shape()의 매개변수를 이용하여 입력의 크기 지정  
- 뉴런의 개수는 경험상!  
- 은닉층 뉴런의 개수 >>> 출력층 뉴런의 개수  
 --> 그래야 충분한 학습이 가능하므로!  

In [3]:
# 은닉층 생성 --> 시그모이드 함수 사용
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # (뉴런의 개수, 활성화 함수 종류 지정, 입력의 크기 지정 --> 첫 번째 층!!)

# 출력층 생성 --> 소프트맥스 함수 사용
dense2 = keras.layers.Dense(10, activation='softmax') # 10개의 클래스를 분류하므로 뉴런의 개수 --> 10개 

**심층 신경망 생성**

In [4]:
model = keras.Sequential([dense1, dense2]) # 만들고 싶은 층을 리스트로 전달 --> 츨력층을 가장 마지막에 둬야함! 

**층에 대한 정보 출력**
- 층 이름(클래스) / 출력 크기 / 파라미터 개수  
 1. 층 이름 --> 변수 이름X -> name 매개변수로 이름 지정 가능! (케라스가 임의로 dense라고 지정)  
 2. 출력 크기  
  (1) None --> 샘플 개수를 고정하지 않고, 어떤 배치 크기에도 유연하게 대응 할 수 있도록!  
  (2) 100, 10 --> 뉴런 개수  
   (샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축됨)  
- keras fit() --> 훈련 데이터 주입 -> 데이터를 잘게 나눔 -> 여러 번에 걸쳐 경사 하강법 진행  
 (미니배치 경사 하강법)  
- 파라미터 개수 --> 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치  

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**은닉층과 출력층 생성(2)**

In [6]:
model = keras.Sequential([
                          keras.layers.Dense(100, activation='sigmoid', input_shape=(784, ), name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')],
                         name='패션 MNIST 모델')

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**은닉층과 출력층 생성(3)**

In [8]:
model = keras.Sequential()

model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

**모델 훈련**

In [9]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5641 - accuracy: 0.8065
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4075 - accuracy: 0.8533
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3726 - accuracy: 0.8658
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3521 - accuracy: 0.8712
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3345 - accuracy: 0.8788


### 3) 은닉층과 출력층 생성
**Flatten 층을 이용한 배열 재정렬**

In [10]:
model = keras.Sequential()
# Flatten() --> 배치 차원을 제외하고, 나머지 입력 차원을 모두 일렬로 펼치는 역할!
# layers를 이용하여 생성하지만, 학습하는 층이 아니므로 깊이가 3인 신경망이라고 하지XX
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**Flatten 층을 이용하여 reshape의 역할을 대신했으므로 reshape() 빼고 변수 재선언**

In [12]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

**Flatten 층을 추가한 뒤, 모델 재훈련**

In [13]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5232 - accuracy: 0.8148
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3892 - accuracy: 0.8599
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3551 - accuracy: 0.8726
Epoch 4/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3361 - accuracy: 0.8796
Epoch 5/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3189 - accuracy: 0.8867


### 4) 검증 세트 성능 확인

In [14]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3737 - accuracy: 0.8698


[0.37368038296699524, 0.8697500228881836]

**adam 옵티마이저를 이용한 모델 훈련**

In [15]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

**optimizer='adam'**  
 --> 적응적 학습률 옵티마이저 -> adam

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5220 - accuracy: 0.8201
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3896 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3496 - accuracy: 0.8728
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3218 - accuracy: 0.8824
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3059 - accuracy: 0.8870


In [17]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3261 - accuracy: 0.8816


[0.326110303401947, 0.8815833330154419]